<a href="https://colab.research.google.com/github/cdbm/F1DataAnalysis/blob/master/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação 
o seguinte notebook foi feito para fazer predições usando modelos de classificação em cima do dataset da fórmula 1. Escolhemos o dataset de resultados, que contém os resultados de cada piloto em todas as corridas, para fazer nossa classificação. Para a classificação escolhemos a columa positions, que define em que posição o piloto terminou a corrida.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
results=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/results.csv")
races=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/races.csv")

Limpando o dataset para ter apenas pilotos que de fato terminaram as corridas.

In [3]:
results = results[results.position != '\\N']
results = results[results.milliseconds != '\\N']

O código a seguir foi feito para tornar mais possível a classificação. Como depois da limpeza restam poucas linhas (aproximadamente 6 mil) e o dataset não tem tantas features que podem ser usadas para o modelo e existem muitas classes possíveis para se predizer (19), resolvemos então transformar as 19 classes que seriam cada posição de chegada em classes mais abrangentes. O nosso modelo agora busca classificar entre as quatro classes de posições que um piloto pode chegar, entre os 5 primeiros, entre o 6º e o 10º, entre o 11º e o 15º e entre o 16º e 19º.

In [4]:
results.position = results.position.astype('int64')

In [5]:
results.loc[results['position'] <= 5, 'beans'] = 'TOP5'
results.loc[(results['position'] > 5) & (results['position'] <= 10) , 'beans'] = 'SIX_TO_TEN'
results.loc[(results['position'] > 10) & (results['position'] <= 15) , 'beans'] = 'EL_TO_FIFTEEN'
results.loc[(results['position'] > 15) & (results['position'] <= 20) , 'beans'] = 'SIXTEEN_TO_TWENTY'

Adicionando a informação de qual circuito foi o dado resultado


In [6]:
results["circuitId"] = results.raceId.map(races.set_index('raceId')['circuitId'])

In [7]:
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,beans,circuitId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,TOP5,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,TOP5,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,TOP5,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,TOP5,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,TOP5,1


Dropando features que ou não podem ser usadas, ou não são úteis ou são úteis até demais para o modelo.

In [8]:
results = results.drop('number', axis=1)
results = results.drop('statusId', axis=1)
results = results.drop('time', axis=1)
results = results.drop('fastestLapTime', axis=1)
results = results.drop('positionOrder', axis=1)
results = results.drop('positionText', axis=1)
results = results.drop('resultId', axis=1)
results = results.drop('fastestLapSpeed', axis=1)
results = results.drop('rank', axis=1)
results = results.drop('fastestLap',axis=1)

In [9]:
results = results.drop('position', axis=1)

Definição de tipos

In [10]:
results.milliseconds = results.milliseconds.astype('int64')
results.beans = results.beans.astype('category')

In [11]:
results.dtypes

raceId              int64
driverId            int64
constructorId       int64
grid                int64
points            float64
laps                int64
milliseconds        int64
beans            category
circuitId           int64
dtype: object

Criação dos Modelos


In [12]:
from sklearn.model_selection import train_test_split
features_results = results.drop('beans', axis=1)
labels_results = results['beans']

In [13]:
features_results

,raceId,driverId,constructorId,grid,points,laps,milliseconds,circuitId
0,18,1,1,1,10.0,58,5690616,1
1,18,2,2,5,8.0,58,5696094,1
2,18,3,3,7,6.0,58,5698779,1
3,18,4,4,11,5.0,58,5707797,1
4,18,5,1,3,4.0,58,5708630,1
...,...,...,...,...,...,...,...,...
24711,1035,20,6,11,0.0,52,4855363,9
24712,1035,832,1,12,0.0,52,4856063,9
24720,1036,1,131,1,25.0,66,5505279,4
24721,1036,830,9,3,18.0,66,5529456,4


In [14]:
labels_results

0                 TOP5
1                 TOP5
2                 TOP5
3                 TOP5
4                 TOP5
             ...      
24711    EL_TO_FIFTEEN
24712    EL_TO_FIFTEEN
24720             TOP5
24721             TOP5
24722             TOP5
Name: beans, Length: 6452, dtype: category
Categories (4, object): ['EL_TO_FIFTEEN', 'SIXTEEN_TO_TWENTY', 'SIX_TO_TEN', 'TOP5']

Divisão do dataset em treinamento validação e teste

In [15]:
feature_train, feature_test, label_train, label_test = train_test_split(features_results, labels_results, test_size=0.2,random_state=1)
feature_train, feature_validation, label_train, label_validation =  train_test_split(feature_train, label_train, test_size=0.25,random_state=1)

In [16]:
print(feature_train.shape)
print(feature_test.shape)
print(feature_validation.shape)

(3870, 8)
(1291, 8)
(1291, 8)


Dicionário para guardar as métricas


In [17]:
stats = {}

Criação, treinamento e avaliação dos modelos

Modelos escolhidos:


*   Naive bayes
*   KNN
*   MLP
*   





In [18]:
from sklearn.metrics import accuracy_score

Naive Bayes

In [19]:
from sklearn.naive_bayes import GaussianNB 
model_naive = GaussianNB()                       
model_naive.fit(feature_train, label_train)                  

prediction_test = model_naive.predict(feature_test)
prediction_train = model_naive.predict(feature_train)   

stats['naive_bayes_test'] = accuracy_score(label_test, prediction_test)
stats['naive_bayes_train'] = accuracy_score(label_train, prediction_train)

print('precisão nos casos de teste: ' + str(stats['naive_bayes_test']))
print('precisão nos casos de treino: ' + str(stats['naive_bayes_train']))

precisão nos casos de teste: 0.6893880712625872
precisão nos casos de treino: 0.6545219638242894


KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=4)             
model_knn.fit(feature_train, label_train) 

prediction_test = model_knn.predict(feature_test)
prediction_train = model_knn.predict(feature_train)   

stats['knn_test'] = accuracy_score(label_test, prediction_test)
stats['knn_train'] = accuracy_score(label_train, prediction_train)

print('precisão nos casos de teste: ' + str(stats['knn_test']))
print('precisão nos casos de treino: ' + str(stats['knn_train']))

precisão nos casos de teste: 0.5894655305964369
precisão nos casos de treino: 0.7423772609819121


MLP


In [21]:
from sklearn.neural_network import MLPClassifier
model_mlp = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='sgd',random_state=1)
model_mlp.fit(feature_train, label_train) 

prediction_test = model_mlp.predict(feature_test)
prediction_train = model_mlp.predict(feature_train)   

stats['mlp_test'] = accuracy_score(label_test, prediction_test)
stats['mlp_train'] = accuracy_score(label_train, prediction_train)

print('precisão nos casos de teste: ' + str(stats['mlp_test']))
print('precisão nos casos de treino: ' + str(stats['mlp_train']))

precisão nos casos de teste: 0.6901626646010844
precisão nos casos de treino: 0.6591731266149871


Random Forrest

In [32]:
from sklearn.ensemble import RandomForestClassifier
model_forrest = RandomForestClassifier(n_estimators=1000)
model_forrest.fit(feature_train, label_train) 

prediction_test = model_forrest.predict(feature_test)
prediction_train = model_forrest.predict(feature_train)   

stats['forrest_test'] = accuracy_score(label_test, prediction_test)
stats['forrest_train'] = accuracy_score(label_train, prediction_train)

print('precisão nos casos de teste: ' + str(stats['forrest_test']))
print('precisão nos casos de treino: ' + str(stats['forrest_train']))


precisão nos casos de teste: 0.9752130131680867
precisão nos casos de treino: 1.0


In [23]:
stats

{'knn_test': 0.5894655305964369,
 'knn_train': 0.7423772609819121,
 'mlp_test': 0.6901626646010844,
 'mlp_train': 0.6591731266149871,
 'naive_bayes_test': 0.6893880712625872,
 'naive_bayes_train': 0.6545219638242894,
 'tree_test': 0.9581719597211464,
 'tree_train': 1.0}